# Advanced RAG 시스템 구현 강의자료

## 1. Multi-Query RAG 기법 소개

Multi-Query RAG는 하나의 질문을 여러 형태로 변환하여 더 포괄적인 검색을 수행하는 기법입니다.

### 기본 원리


원본 질문 → 7개의 변형 질문 생성 → 각각 검색 → 중복 제거 → Context Compression → 답변

### 장점

- **검색 범위 확대**: 다양한 표현으로 더 많은 관련 문서 발견
- **정확도 향상**: 단일 질문의 한계를 보완
- **누락 최소화**: 동의어나 유사 표현 문서까지 검색
- **노이즈 감소**: Context Compression으로 불필요한 정보 제거
- **효율성 증대**: 압축으로 토큰 사용량 최적화

## 2. 필요한 라이브러리

In [1]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate as LCPromptTemplate
from langchain_ollama import OllamaEmbeddings, OllamaLLM

## 3. 시스템 구성 요소

### 3.1 벡터 데이터베이스  저장

- 벡터 데이터베이스를 로컬에 저장하여 재사용 가능
- 매번 임베딩을 다시 생성할 필요 없어 시간 절약

In [2]:
VECTOR_DB_PATH = "faiss_index_MQ_RAG"

### 3.2 벡터 DB 생성 함수

- 청크 크기 1200자로 설정하여 충분한 컨텍스트 제공
- 오버랩 200자로 정보 손실 방지
- PDF 문서 직접 처리 가능

In [3]:
def create_vector_db():
    # PDF 문서 로딩
    loader = PyPDFLoader("./document/원자력안전관리_설명자료.pdf")
    docs = loader.load()
    
    # 문서 분할 (청크 크기: 1200, 오버랩: 200)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200, 
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(docs)
    
    # 임베딩 및 벡터 저장소 생성
    embeddings = OllamaEmbeddings(model="exaone3.5:7.8b")
    vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)
    
    # 로컬에 저장
    vector_store.save_local(VECTOR_DB_PATH)
    return vector_store

### 3.3 벡터 DB 로딩 또는 생성

**장점:**
- 첫 실행 후 빠른 시작 가능
- 리소스 효율적 사용

In [4]:
if os.path.exists(VECTOR_DB_PATH):
    print("기존 벡터 DB를 로드합니다.")
    embeddings = OllamaEmbeddings(model="exaone3.5:7.8b")
    vector_store = FAISS.load_local(
        VECTOR_DB_PATH,
        embeddings,
        allow_dangerous_deserialization=True
    )
else:
    print("새로운 벡터 DB를 생성합니다.")
    vector_store = create_vector_db()

새로운 벡터 DB를 생성합니다.


## 4. 프롬프트 설계

### 4.1 QA 프롬프트

In [5]:
qa_prompt = PromptTemplate.from_template(
"""당신은 질문-답변(Question-Answering)을 수행하는 AI 어시스턴트입니다. 
당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.

검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 
만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 
"주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다"라고 답하세요.

질문과 관련성이 높은 내용만 답변하고 추측된 내용을 생성하지 마세요. 
기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: {question}
#Context: {context}
#Answer:"""
)

### 4.2 Multi-Query 생성 프롬프트

**목적:**
- 하나의 질문을 다양한 표현으로 변환
- 검색 범위 확대로 더 많은 관련 문서 발견


In [6]:
multi_query_prompt = LCPromptTemplate.from_template(
"""주어진 사용자 질문의 다양한 버전을 생성하는 AI입니다.
사용자의 질문을 paraphrasing해서 질문의 의도와 의미가 동일한 새로운 질문 7개를 만들어냅니다.
질문 속 핵심 단어는 유지하고 조사나 수식어와 같은 부가적인 표현을 paraphrasing합니다.

각 질문은 다음과 같은 다양한 표현 방식을 사용하세요:
1. 정의를 묻는 형태
2. 설명을 요청하는 형태  
3. 구체적인 내용을 묻는 형태
4. 절차나 과정을 묻는 형태
5. 특징이나 특성을 묻는 형태
6. 목적이나 이유를 묻는 형태
7. 다른 용어로 표현한 형태

질문: {question}

7가지 다양한 질문:"""
)

Context Compression 프롬프트

**목적:**
- 검색된 여러 문서들에서 핵심 정보만 추출
- 중복 내용 제거 및 노이즈 감소
- 질문과 관련된 정보만 선별적으로 유지

In [7]:
context_compression_prompt = LCPromptTemplate.from_template(
"""당신은 문서 요약 전문가입니다. 주어진 여러 문서들에서 질문과 관련된 핵심 정보만을 추출하여 간결하게 요약해주세요.

중복된 내용은 제거하고, 질문에 답하는데 필요한 가장 중요한 정보들만 포함하세요.
요약된 내용은 원본의 의미를 보존하면서도 불필요한 세부사항은 제거해야 합니다.

질문: {question}

문서들:
{documents}

압축된 컨텍스트:"""
)

## 4.3 체인 및 Retriever 설정

In [8]:
# LLM 설정
llm = OllamaLLM(model="exaone3.5:7.8b", temperature=0)

# QA 체인 정의
qa_chain = qa_prompt | llm | StrOutputParser()

# Context Compression 체인 정의
compression_chain = context_compression_prompt | llm | StrOutputParser()

# Basic Retriever
basic_retriever = vector_store.as_retriever()

## 5. Advanced RAG 구현

### 5.1 Multi-Query 생성 함수

**핵심 특징:**
- 7개의 다양한 변형 질문 생성
- 번호나 특수문자 자동 제거
- 품질 필터링으로 의미있는 질문만 선별

In [9]:
def generate_multiple_queries(question, llm, num_queries=7):
    """여러 개의 변형 질문을 생성하는 함수"""
    prompt_input = multi_query_prompt.format(question=question)
    generated_text = llm.invoke(prompt_input)
    
    # 생성된 텍스트에서 질문들 추출
    generated_queries = []
    lines = generated_text.split("\n")
    
    for line in lines:
        line = line.strip()
        if line and not line.endswith(":") and line != "7가지 다양한 질문:":
            # 번호나 특수문자 제거
            cleaned_line = line.lstrip("1234567890.-•*").strip()
            if cleaned_line and len(cleaned_line) > 10:  # 너무 짧은 텍스트 제외
                generated_queries.append(cleaned_line)
    
    # 최대 num_queries개까지만 반환
    return generated_queries[:num_queries]

### 5.2 Context Compression 함수

**핵심 특징:**
- 여러 문서를 하나로 결합
- LLM을 통한 지능적 압축
- 원본 의미 보존하면서 노이즈 제거


In [10]:
def compress_context(question, documents, llm):
    """검색된 문서들을 압축하는 함수"""
    if not documents:
        return ""
    
    # 문서들을 하나의 텍스트로 결합
    combined_docs = "\n\n---문서구분---\n\n".join([doc.page_content for doc in documents])
    
    # 컨텍스트 압축 실행
    compressed_context = compression_chain.invoke({
        "question": question,
        "documents": combined_docs
    })
    
    return compressed_context

### 5.3 Advanced 질문 생성 및 검색 함수

**핵심 특징:**
- Multi-Query 접근법으로 검색 품질 향상
- 중복 제거로 효율성 증대
- Context Compression으로 노이즈 감소
- 예외 처리로 안정성 확보

In [11]:
def generate_advanced_context_with_compression(question, llm, vector_store, num_queries=7):
    """개선된 Multi-Query RAG with Context Compression"""
    
    # 1. 여러 변형 질문 생성
    generated_queries = generate_multiple_queries(question, llm, num_queries)
    
    print(f"\n[생성된 {len(generated_queries)}개의 변형 질문들]")
    for i, q in enumerate(generated_queries, 1):
        print(f"{i}. {q}")
    
    # 2. 각 질문으로 검색 후 통합
    print(f"\n[Multi-Query 검색 실행]")
    all_docs = []
    for i, q in enumerate(generated_queries, 1):
        search_results = vector_store.similarity_search(q, k=3)  # 각 쿼리당 최대 3개 문서
        if search_results:
            all_docs.extend(search_results)
            print(f"질문 {i}: {len(search_results)}개 문서 검색됨")
    
    # 3. 중복 문서 제거
    unique_docs = []
    seen = set()
    for doc in all_docs:
        if doc.page_content not in seen:
            unique_docs.append(doc)
            seen.add(doc.page_content)
    
    print(f"총 {len(all_docs)}개 문서 검색, 중복 제거 후 {len(unique_docs)}개 고유 문서")
    
    # 4. Context Compression 적용
    if unique_docs:
        print("\n[Context Compression 실행 중...]")
        original_length = len(''.join([doc.page_content for doc in unique_docs]))
        compressed_context = compress_context(question, unique_docs, llm)
        compressed_length = len(compressed_context)
        
        compression_ratio = round((1 - compressed_length/original_length) * 100, 1) if original_length > 0 else 0
        print(f"압축 완료: {original_length} → {compressed_length} 문자 (압축률: {compression_ratio}%)")
        return compressed_context
    else:
        print("관련 문서를 찾을 수 없습니다.")
        return ""

# 6. Basic RAG vs Advanced RAG with Compression 비교

### 6.1 Basic RAG

**특징:**
- 빠른 검색 속도
- 단순한 구조
- 제한적인 검색 범위
- 단일 질문으로만 검색

### 6.2 Multi-Query RAG with Context Compression

**특징:**
- 더 포괄적인 검색 (7개 변형 질문)
- 높은 정확도
- Context Compression으로 노이즈 제거
- 효율적인 토큰 사용
- 다소 느린 속도 (LLM 호출 추가)

### 6.3 성능 개선 효과

**검색 품질:**
- 7개 변형 질문으로 검색 범위 확대
- 동의어, 유사 표현까지 포괄하는 검색

**효율성:**
- Context Compression으로 불필요한 정보 제거
- 압축률 표시로 성능 모니터링 가능

**정확성:**
- 중복 문서 자동 제거
- 질문과 관련된 핵심 정보만 선별


## 7. 실행 예시

In [12]:
while True:
    question = input("\n\n질문을 입력하세요 ('끝' 입력 시 종료): ")
    if question.strip().lower() in ["끝", "exit", "quit"]:
        break

    print("\n" + "="*80)
    
    # BASIC RAG 실행
    print("\n[ BASIC RAG 결과 ]")
    basic_docs = basic_retriever.invoke(question)
    basic_context = "\n\n".join(doc.page_content for doc in basic_docs)
    formatted_basic = {"context": basic_context, "question": question}
    for chunk in qa_chain.stream(formatted_basic):
        print(chunk, end="", flush=True)

    print("\n\n" + "-"*60)
    
    # ADVANCED RAG with Compression 실행
    print("\n[ ADVANCED RAG with Compression 결과 ]")
    compressed_context = generate_advanced_context_with_compression(
        question, llm, vector_store, num_queries=7
    )
    
    if compressed_context:
        formatted_advanced = {"context": compressed_context, "question": question}
        print("\n답변: ", end="")
        for chunk in qa_chain.stream(formatted_advanced):
            print(chunk, end="", flush=True)
    else:
        print("\n답변: 주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.")

    print("\n\n" + "="*80)



질문을 입력하세요 ('끝' 입력 시 종료):  방사선 비상이 뭐야?




[ BASIC RAG 결과 ]
주어진 정보에서 방사선 비상에 대한 직접적인 정의나 설명은 제공되지 않았습니다. 따라서, 주어진 문맥에서 질문에 대한 정확한 답변을 제공할 수 없습니다.

**답변:** 주어진 정보로는 방사선 비상에 대한 구체적인 설명이 포함되어 있지 않습니다. 방사선 비상은 일반적으로 방사성 물질의 누출이나 사고로 인해 사람이나 환경에 방사선 노출 위험이 발생한 상황을 의미하지만, 이 문맥에서는 해당 개념에 대한 자세한 내용을 찾을 수 없습니다. 다른 관련 자료를 참조하시는 것이 좋을 것 같습니다.

------------------------------------------------------------

[ ADVANCED RAG with Compression 결과 ]

[생성된 7개의 변형 질문들]
1. **정의를 묻는 형태**: 방사선 비상 상황이란 무엇을 의미하나요?
2. **설명을 요청하는 형태**: 방사선 비상이 발생했을 때 어떤 상황이 펼쳐지는지 설명해 주실 수 있나요?
3. **구체적인 내용을 묻는 형태**: 방사선 비상 상황에서 주의해야 할 주요 사항들은 무엇인가요?
4. **절차나 과정을 묻는 형태**: 방사선 비상이 선포되었을 때 따라야 하는 주요 대응 절차는 무엇인가요?
5. **특징이나 특성을 묻는 형태**: 방사선 비상의 주요 특징과 그 위험성은 무엇인가요?
6. **목적이나 이유를 묻는 형태**: 방사선 비상 선포의 주된 목적과 필요성은 무엇인가요?
7. **다른 용어로 표현한 형태**: 핵 방사선 사고 시 취해야 할 조치와 그 배경은 어떻게 설명할 수 있나요?

[Multi-Query 검색 실행]
질문 1: 3개 문서 검색됨
질문 2: 3개 문서 검색됨
질문 3: 3개 문서 검색됨
질문 4: 3개 문서 검색됨
질문 5: 3개 문서 검색됨
질문 6: 3개 문서 검색됨
질문 7: 3개 문서 검색됨
총 21개 문서 검색, 중복 제거 후 5개 고유 문서

[Context Compression 실행 중...]
압축 완



질문을 입력하세요 ('끝' 입력 시 종료):  끝


# 실습: 질문 변형 방식의 변화와 효과 비교
**생성 질문의 형태를 변경하고 차이를 비교해보자**  

하나의 단일 쿼리를 입력받아 여러 개의 변형 질문으로 생성할 때, 앞서서 변형한 질문 대신 다른 변형방식을 적용해보고 차이를 비교해보는 멀티 쿼리 실습입니다. 
서로 다른 변형 방식을 적용해보고, 어떤 결과가 나오는지 직접 경험해보면서 최적의 질문 생성 전략을 찾아봅시다.

In [ ]:
#프롬포트 수정
multi_query_prompt = LCPromptTemplate.from_template(
"""주어진 사용자 질문의 다양한 버전을 생성하는 AI입니다.
사용자의 질문을 paraphrasing해서 질문의 의도와 의미가 동일한 새로운 질문 7개를 만들어냅니다.
질문 속 핵심 단어는 유지하고 조사나 수식어와 같은 부가적인 표현을 paraphrasing합니다.

각 질문은 다음과 같은 다양한 표현 방식을 사용하세요:                



질문: {question}

7가지 다양한 질문:"""
)


In [ ]:
while True:
    question = input("\n\n질문을 입력하세요 ('끝' 입력 시 종료): ")
    if question.strip().lower() in ["끝", "exit", "quit"]:
        break

    print("\n" + "="*80)
    
    # BASIC RAG 실행
    print("\n[ BASIC RAG 결과 ]")
    basic_docs = basic_retriever.invoke(question)
    basic_context = "\n\n".join(doc.page_content for doc in basic_docs)
    formatted_basic = {"context": basic_context, "question": question}
    for chunk in qa_chain.stream(formatted_basic):
        print(chunk, end="", flush=True)

    print("\n\n" + "-"*60)
    
    # ADVANCED RAG with Compression 실행
    print("\n[ ADVANCED RAG with Compression 결과 ]")
    compressed_context = generate_advanced_context_with_compression(
        question, llm, vector_store, num_queries=7
    )
    
    if compressed_context:
        formatted_advanced = {"context": compressed_context, "question": question}
        print("\n답변: ", end="")
        for chunk in qa_chain.stream(formatted_advanced):
            print(chunk, end="", flush=True)
    else:
        print("\n답변: 주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.")

    print("\n\n" + "="*80)